<a href="https://colab.research.google.com/github/fhswf/ki-wir/blob/main/VAEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Autoencoder


In [10]:
# Setup on Colab
!pip install gradio &> /dev/null
!pip install pytorch_lightning &> /dev/null
!if [ ! -e ki_wir ]; then git clone https://github.com/fhswf/ki-wir.git ki_wir; else cd ki_wir; git pull; fi

Already up to date.


In [6]:
import gradio as gr
import torch
import numpy as np
import ki_wir.models.vanilla_vae as vae

from PIL import Image
from torchvision import transforms

In [9]:
device = torch.device("cuda:0")
model = vae.VanillaVAE(in_channels=3, latent_dim=128).to(device)
model.load_state_dict(torch.load("ki_wir/pretrained/vanilla.ckpt", map_location=device))

def reconstruct(original):
  img = Image.fromarray(original)
  img = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        ])(img)

  orig = transforms.ToPILImage(mode='RGB')(img)
  img = torch.unsqueeze(img.cuda(), 0)
  enc = model.encode(img)
  dec = model.decode(enc)
  dec = torch.squeeze(dec, 0)

  return [orig, transforms.ToPILImage(mode='RGB')(dec)]

ModuleNotFoundError: ignored

In [ ]:
out1 = gr.outputs.Image(type="auto", label="original")
out2 = gr.outputs.Image(type="auto", label="reconstructed")
iface = gr.Interface(fn=reconstruct, inputs="image", outputs=[out1, out2]).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://45829.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app
